Last Update @ 2020.12.04

- Huggingface Transformers 4.0.0  버전 반영

# Package 설치 & 데이터 받기

In [ ]:
!pip install pytorch_lightning==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.8/795.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.4/136.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=a6ccff511135805b4eef30aa9b774d0952391e4ce2691e956d4cd86d7114d7a6
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5d

In [ ]:
!pip install -q transformers soynlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00


In [ ]:
!pip install emoji==1.6.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170282 sha256=4bf2019653b1cb7968fc0a278a54930bd20ab253c0d83e3217bf7062b96cf9cd
  Stored in directory: /root/.cache/pip/wheels/83/42/4a/2e5a7b28b80e3a4f143cb5bbfcfbfe54958ccb533bb8ce2a99
Successfully built emoji


# 패키지 import & 기본 Args 설정

In [ ]:
import os
import pandas as pd

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import BertForSequenceClassification, BertTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
tr = pd.read_excel("/content/drive/MyDrive/유튜브 도덕판단 연구용/mj_sentiment_train.xlsx")
ts = pd.read_excel("/content/drive/MyDrive/유튜브 도덕판단 연구용/mj_sentiment_test.xlsx")

In [ ]:
tr_mini = tr[["comment","real_label"]].sample(n = 12000)
ts_mini = ts[["comment","real_label"]].sample(n = 3000)

In [ ]:
tr_mini.to_excel("/content/drive/MyDrive/tr_mini.xlsx")
ts_mini.to_excel("/content/drive/MyDrive/ts_mini.xlsx")


## 기본 학습 Arguments

In [ ]:
#beomi/kcbert-large

class Arg:
    random_seed: int = 42  # Random Seed
    pretrained_model: str = 'beomi/kcbert-base'  # Transformers PLM name,
    pretrained_tokenizer: str = ''  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    auto_batch_size: str = 'power'  # Let PyTorch Lightening find the best batch size
    batch_size: int = 32  # Optional, Train/Eval Batch Size. Overrides `auto_batch_size`
    lr: float = 5e-6  # Starting Learning Rate
    epochs: int = 1  # Max Epochs
    max_length: int = 150  # Max Length input size
    report_cycle: int = 100  # Report (Train Metrics) Cycle
    train_data_path: str = "/content/drive/MyDrive/tr_mini.xlsx"  # Train Dataset file /content/drive/MyDrive/mj_sentiment_train.xlsx
    val_data_path: str = "/content/drive/MyDrive/ts_mini.xlsx"  # Validation Dataset file /content/drive/MyDrive/mj_sentiment_test.xlsx
    cpu_workers: int = os.cpu_count()  # Multi cpu workers
    test_mode: bool = False  # Test Mode enables `fast_dev_run`
    optimizer: str = 'AdamW'  # AdamW vs AdamP
    lr_scheduler: str = 'exp'  # ExponentialLR vs CosineAnnealingWarmRestarts
    fp16: bool = False  # Enable train on FP16
    tpu_cores: int = 0  # Enable TPU with 1 core or 8 cores

args = Arg()

## 기본값을 Override 하고싶은 경우 아래와 같이 수정

In [ ]:
!nvidia-smi

위에서 GPU가 V100/P100이면 아래 `batch_size`  를 32 이상으로 하셔도 됩니다.

In [ ]:
# args.tpu_cores = 8  # Enables TPU
args.fp16 = True  # Enables GPU FP16
args.batch_size = 32  # Force setup batch_size

# Model 만들기 with Pytorch Lightning

In [ ]:
class Model(LightningModule):
    def __init__(self, options):
        super().__init__()
        self.args = options
        self.bert = BertForSequenceClassification.from_pretrained(self.args.pretrained_model)
        self.tokenizer = BertTokenizer.from_pretrained(
            self.args.pretrained_tokenizer
            if self.args.pretrained_tokenizer
            else self.args.pretrained_model
        )

    def forward(self, *args, **kwargs):
        return self.bert(*args, **kwargs)


    def training_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = labels.cpu().numpy()
        y_pred = preds.cpu().numpy()

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'train_loss': loss.cpu().detach().numpy().tolist(),
            'train_acc': metrics[0],
            'train_precision': metrics[1],
            'train_recall': metrics[2],
            'train_f1': metrics[3],
        }
        if (batch_idx % self.args.report_cycle) == 0:
            print()
            pprint(tensorboard_logs)
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        data, _ = batch
        logits = self(input_ids=data)
        preds = logits.logits.argmax(dim=-1)
        return preds.cpu().numpy()


    def trainining_epoch_end(self, outputs):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        _loss = loss / len(outputs)

        loss = float(_loss)
        y_true = []
        y_pred = []

        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'val_loss': loss,
            'val_acc': metrics[0],
            'val_precision': metrics[1],
            'val_recall': metrics[2],
            'val_f1': metrics[3],
        }

        print()
        pprint(tensorboard_logs)
        return {'loss': _loss, 'log': tensorboard_logs}




    def validation_epoch_end(self, outputs):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        _loss = loss / len(outputs)

        loss = float(_loss)
        y_true = []
        y_pred = []

        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'val_loss': loss,
            'val_acc': metrics[0],
            'val_precision': metrics[1],
            'val_recall': metrics[2],
            'val_f1': metrics[3],
        }

        print()
        pprint(tensorboard_logs)
        return {'loss': _loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        if self.args.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.args.lr)
        elif self.args.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.args.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.args.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.args.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def read_data(self, path):
        if path.endswith('xlsx'):
            return pd.read_excel(path)
        elif path.endswith('csv'):
            return pd.read_csv(path)
        elif path.endswith('tsv') or path.endswith('txt'):
            return pd.read_csv(path, sep='\t')
        else:
            raise NotImplementedError('Only Excel(xlsx)/Csv/Tsv(txt) are Supported')

    def preprocess_dataframe(self, df):
        emojis = ''.join(emoji.UNICODE_EMOJI.keys())
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

        def clean(x):
            x = pattern.sub(' ', x)
            x = url_pattern.sub('', x)
            x = x.strip()
            x = repeat_normalize(x, num_repeats=2)
            return x

        df['comment'] = df['comment'].map(lambda x: self.tokenizer.encode(
            clean(str(x)),
            padding='max_length',
            max_length=self.args.max_length,
            truncation=True,
        ))
        return df

    def train_dataloader(self):
        df = self.read_data(self.args.train_data_path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['comment'].to_list(), dtype=torch.long),
            torch.tensor(df['real_label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=True,
            num_workers=self.args.cpu_workers,
        )

    def val_dataloader(self):
        df = self.read_data(self.args.val_data_path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['comment'].to_list(), dtype=torch.long),
            torch.tensor(df['real_label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=False,
            num_workers=self.args.cpu_workers,
        )


In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filename='epoch{epoch}-val_acc{val_acc:.4f}',
    monitor='val_acc',
    save_top_k=3,
    mode='max',
    auto_insert_metric_name=False,
)

# 학습!

> 주의: 1epoch별로 GPU-P100기준 약 1-2시간, GPU V100기준 ~30분이 걸립니다.

> Update @ 2020.09.01
> 최근 Colab Pro에서 V100이 배정됩니다.

```python
# 1epoch 기준 아래 score가 나옵니다.
{'val_acc': 0.90522,
 'val_f1': 0.9049023739289227,
 'val_loss': 0.23429009318351746,
 'val_precision': 0.9143146796431468,
 'val_recall': 0.8956818813808446}
```

In [ ]:
#main()
# 시도 1회차

In [ ]:
print("Using PyTorch Ver", torch.__version__)
print("Fix Seed:", args.random_seed)
seed_everything(args.random_seed)
model = Model(args)

print(":: Start Training ::")
trainer = Trainer(
max_epochs=args.epochs,
fast_dev_run=args.test_mode,
num_sanity_val_steps=None if args.test_mode else 0,
auto_scale_batch_size=args.auto_batch_size if args.auto_batch_size and not args.batch_size else False,

# For GPU Setup
deterministic=torch.cuda.is_available(),
gpus=-1 if torch.cuda.is_available() else None,
precision=16 if args.fp16 else 32,
# For TPU Setup
# tpu_cores=args.tpu_cores if args.tpu_cores else None,
    )


INFO:lightning_lite.utilities.seed:Global seed set to 42


Using PyTorch Ver 2.0.1+cu118
Fix Seed: 42


Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly i

:: Start Training ::


In [ ]:
trainer.fit(model) # all train file, epoch = 1, batch = 32

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:382: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type                          | Params
-------------------------------------------------------
0 | bert | BertForSequenceClassification | 108 M 
-------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
217.840   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



{'train_acc': 0.4375,
 'train_f1': 0.0,
 'train_loss': 0.7284774780273438,
 'train_precision': 0.0,
 'train_recall': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin


{'train_acc': 0.8125,
 'train_f1': 0.8125000000000001,
 'train_loss': 0.4157676696777344,
 'train_precision': 1.0,
 'train_recall': 0.6842105263157895}

{'train_acc': 0.65625,
 'train_f1': 0.6857142857142857,
 'train_loss': 0.5887413024902344,
 'train_precision': 0.7058823529411765,
 'train_recall': 0.6666666666666666}

{'train_acc': 0.8125,
 'train_f1': 0.8333333333333333,
 'train_loss': 0.420137882232666,
 'train_precision': 0.8823529411764706,
 'train_recall': 0.7894736842105263}


Validation: 0it [00:00, ?it/s]


{'val_acc': 0.8126666666666666,
 'val_f1': 0.7933823529411763,
 'val_loss': 0.40578529238700867,
 'val_precision': 0.8858784893267652,
 'val_recall': 0.7183754993342211}


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
trainer.validate(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]


{'val_acc': 0.8605625,
 'val_f1': 0.8544303797468353,
 'val_loss': 0.3021318316459656,
 'val_precision': 0.9014869888475836,
 'val_recall': 0.8120426640208359}


[{}]

# 라벨 뽑기

In [ ]:
predictions_tr = trainer.predict(dataloaders=model.train_dataloader())

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:129: UserWarning: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.predict(ckpt_path='best')` to use the best model or `.predict(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_1/checkpoints/epoch=0-step=375.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/lightning_logs/version_1/checkpoints/epoch=0-step=375.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `predict_dataloader`'s sampler has shuffling enabled, it is stron

Predicting: 375it [00:00, ?it/s]

In [ ]:
predictions_ts = trainer.predict(dataloaders=model.val_dataloader())

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:129: UserWarning: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.predict(ckpt_path='best')` to use the best model or `.predict(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_4/checkpoints/epoch=0-step=3750.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/lightning_logs/version_4/checkpoints/epoch=0-step=3750.ckpt


Predicting: 3750it [00:00, ?it/s]

In [ ]:
predictions_tr

In [ ]:
tr_labels = []
for sublist in predictions_tr:
    tr_labels.extend(sublist)

print(tr_labels)
print(len(tr_labels))

[1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 

In [ ]:
ts_labels = []
for sublist in predictions_ts:
    ts_labels.extend(sublist)

print(ts_labels)
print(len(ts_labels))

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 

In [ ]:
import pandas as pd
tr = pd.read_excel("/content/drive/MyDrive/mj_sentiment_train.xlsx")
ts = pd.read_excel("/content/drive/MyDrive/mj_sentiment_test.xlsx")

tr = tr[["comment","real_label"]]
ts = ts[["comment","real_label"]]

tr["kcbert_label"] = tr_labels
ts["kcbert_label"] = ts_labels

In [ ]:
tr

,comment,real_label,kcbert_label
0,형 딸도 문제가 많다던데,0,1
1,냥이 차칸척 인분 추가요 온데 가르치고다니네 별,0,1
2,너무 물러터진 대한민국 법 술에는 엄청 관대한 나라 이러니 외국인들도 술먹고 사고치...,0,0
3,한심한 공무원들 이네요 완전 무방비상태네요 국민들은 누굴 믿어야하는지,0,0
4,이걸 어떻게 보이스피싱으로 생각하고 신고를하지 신기하네요 저였으면 그런생각을 아예못...,1,1
...,...,...,...
119995,대한민국 만세 대단한 시민의식입니다 감동했습니다,1,1
119996,드디어 올라가넼,1,1
119997,나 눈물나와서 울었어요 마지막 할아버지 역주행으로 자전거 타고가시는데 자전거메모에 ...,1,0
119998,난 생존수영 배웠지만 년만에 다 까먹어버렸음 저아이는 진짜 대단하다 왠만한 어린아이...,1,0


In [ ]:
ts

,comment,real_label,kcbert_label
0,저딴것들이 우리세금 냠냠하네,1,0
1,요즘 대학교 에타보면 저런 애들이 너무많아서 겁이남 학벌주의에 심취하는건 새내기때에...,0,0
2,자기 딸이 회사에서 밥 짓는다고 그러면 화낼 인간들이 직원에게는 저러고 있네 어휴 ...,0,0
3,테이저건 쏘고 묶어놔야지 승무원들 처벌해라 뭐하는거야,0,0
4,네 사정이야 어찌되었든 과속하셨구요 공권력 사칭으로 간주 벌금 이빠이 되겠습니다 서...,1,0
...,...,...,...
39995,저런데 한국이 중국을 도와줘야 되는가 절대 않되는거지 인권이 심각하다 북한이랑 다를...,0,0
39996,이런분들은 영웅이십니다 국민영웅들입니다 이런분들은 표창을 줘야합니다 그런분들 앞으로...,1,1
39997,정신병있네 똑똑한척,0,0
39998,훌륭한 젊은이 아직 대한민국은 그대와같은 젊은이가 있는한 최고가 될수있다 젊은 영웅...,1,1


In [ ]:
tr.to_excel("/content/drive/MyDrive/kcbert_label_trainset.xlsx")
ts.to_excel("/content/drive/MyDrive/kcbert_label_testnset.xlsx")

In [ ]:
predictions

[array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 1]),
 array([1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 0]),
 array([1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 1]),
 array([0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 0, 1, 1, 0, 0, 1, 1, 1, 0]),
 array([1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
        1, 1, 1, 0, 0, 0, 0, 1, 1, 0]),
 array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 0, 0, 1]),
 array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 0, 

# 새 섹션

In [ ]:
import torch

In [ ]:
# 모델 저장 제발
torch.save(model,'/content/drive/MyDrive/유튜브 도덕판단 연구용/mj_model_kcbert_v5.pt')

# mj_model_kcbert_v5.pt: all train file, epoch = 2 ,batch size = 32, val_acc = 0.88

In [ ]:
# 저장한 모델 불러오기
model = torch.load("/content/drive/MyDrive/유튜브 도덕판단 연구용/mj_model_kcbert_v5.pt")
model.eval()

In [ ]:
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/mj_sentiment_train_small.xlsx")

In [ ]:
x = df["comment"]
x

0                            평상시에 의료진들이 진화 연습을 충실히 한 것 같네요
1                                               역시 어메이징 붓싼
2                                               아들도 형사감인건가
3                                           영웅이 따로 있나 대단하다
4        영상속의 시민영웅님들 존경합니다 그리고 아이 어머니 피곤하시더라도 안전은 챙기세요 ...
                               ...                        
74995                                  아니 왜 남자를 구하는 여자는 없냐
74996                 세상에는 따뜻한 사람들이 여전히 있기에 우리사회가 돌아가는것같네요
74997                                         와 역시 번남들 멋지다
74998    진짜 이런분들땜에 대한민국이 돌아간다 근데 형사님이랑 아드님 얼굴은 가려줘야 하는거...
74999                               인가요 인가요 여윽시 흰색 는 과학이네요
Name: comment, Length: 75000, dtype: object

In [ ]:
import numpy as np

In [ ]:
def infer_sigmoid(x):
    result = torch.sigmoid(model(**model.tokenizer(x, return_tensors='pt')).logits)
    result = result.detach().numpy()
    result = np.argmax(result)
    return result

In [ ]:
infer_sigmoid("평상시에 의료진들이 진화 연습을 충실히 한 것 같네요")

1

In [ ]:
kcbert_base_prediction = model.predict(x[34252])
kcbert_base_prediction

AttributeError: ignored

# LIME with KCBERT

In [ ]:
loaded_model = torch.load("/content/drive/MyDrive/mj_model_kcbert_v5.pt")

NameError: ignored

In [ ]:
y_pred = trainer.predict([ids_test, masks_test, segments_test])

NameError: ignored

In [ ]:
!pip install lime

In [ ]:
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
import numpy as np

In [ ]:
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/유튜브 도덕판단 연구용/mj_sentiment_train_small.xlsx")

In [ ]:
trainer.predict(dataloaders = text_sample)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:129: UserWarning: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.predict(ckpt_path='best')` to use the best model or `.predict(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_1/checkpoints/epoch=0-step=375.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/lightning_logs/version_1/checkpoints/epoch=0-step=375.ckpt


Predicting: 375it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/epoch/prediction_epoch_loop.py:173: UserWarning: Lightning couldn't infer the indices fetched for your dataloader.
  warning_cache.warn("Lightning couldn't infer the indices fetched for your dataloader.")


ValueError: ignored

In [ ]:
import random
id = random.randrange(0,len(df["comment"]))
text_sample = df["comment"][id]
print(text_sample)

촉법소년도 아니네 인생 조져라 피해 보상금 두둑히 챙겨요 피해자분들


In [ ]:
tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")

In [ ]:
def infer_sigmoid(x):
    result = torch.sigmoid(model(**model.tokenizer(x, return_tensors='pt')).logits)
    result = result.detach().numpy()
    label = np.argmax(result)
    return result

In [ ]:
infer_sigmoid(text_sample)

array([[0.763594  , 0.42379284]], dtype=float32)

In [ ]:
from lime.lime_text import LimeTextExplainer
class_names = ["부정","긍정"]
explainer = LimeTextExplainer(class_names=class_names)

exp = explainer.explain_instance(text_sample,
                                 infer_sigmoid)

ValueError: ignored

# SHAP with KCBERT

In [ ]:
!pip install shap

In [ ]:
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')

In [ ]:
model

Model(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30000, 768, padding_idx=0)
        (position_embeddings): Embedding(300, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True

In [ ]:
# define a prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

In [ ]:
comments_set = tr_mini["comment"].reset_index()
comments_set = comments_set[["comment"]]

In [ ]:
comments_set

,comment
0,역시 한국 계집들은 민폐다
1,이제 음주 운전이면 무조건 면허 취소 시키고 취득 할 수 있는 기간도 늘려야 하는 ...
2,음 아무리 화나두 그렇지 뱅기안에서 같이 탄사람들 얼마나 무서웠을고
3,모범택시 하시나
4,애들이 너무 이쁘네요 고사리 같은 손으로 화물차 아저씨 바빠도 한 번 더 확인하세요
...,...
11995,진짜 경찰이네
11996,저거 뒤지면 자연사아니냐
11997,당신말고도 일할 차가 많다 하는 쓰래기같은 생각을 한명이라도 갖고있으니까 상차하는 ...
11998,훌륭하신 시민이네요


In [ ]:
sample_text = comments_set["comment"][11997]
sample_text

'당신말고도 일할 차가 많다 하는 쓰래기같은 생각을 한명이라도 갖고있으니까 상차하는 사람도 같은 생각 아닐까 다같이 거부했다면 과적차는 생기지 않았을것이다 다같이 모여서 데모를 할께 아니라 다같이 담합을 해보는건 어떨까'

In [ ]:
f(sample_text)

RuntimeError: ignored

In [ ]:
shap.plots.bar(shap_values.abs.sum(0))

In [ ]:
def f(x):
    if not isinstance(x, list):
        x = [x]
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    tv = tv[:, :min(tv.shape[1], 300)]
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:, 1])  # use one vs rest logit units
    return val

sample_text = comments_set["comment"][11997]

f(sample_text)


RuntimeError: ignored